# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234691619029                   -0.50    7.0         
  2   -7.250195969086       -1.81       -1.40    1.0   8.10ms
  3   -7.251141098885       -3.02       -2.02    2.0   9.38ms
  4   -7.251134460698   +   -5.18       -1.98    2.0   10.1ms
  5   -7.251328803411       -3.71       -2.62    1.0   7.88ms
  6   -7.251338277453       -5.02       -3.20    1.0   8.30ms
  7   -7.251338728863       -6.35       -3.64    2.0   9.55ms
  8   -7.251338791050       -7.21       -4.11    2.0   10.2ms
  9   -7.251338798136       -8.15       -4.89    1.0   8.29ms
 10   -7.251338798634       -9.30       -5.08    3.0   11.7ms
 11   -7.251338798696      -10.20       -5.66    1.0   8.40ms
 12   -7.251338798703      -11.21       -5.95    3.0   11.8ms
 13   -7.251338798704      -11.76       -6.57    2.0   10.1ms
 14   -7.251338798705      -13.35       -6.94    1.0   8.64ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04890906455687379
[ Info: Arnoldi iteration step 2: normres = 0.6751745203226955
[ Info: Arnoldi iteration step 3: normres = 0.8277639929120804
[ Info: Arnoldi iteration step 4: normres = 0.3521866698687679
[ Info: Arnoldi iteration step 5: normres = 0.4089757863559144
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.69e-02, 7.88e-02, 2.89e-01, 2.72e-01, 4.81e-02)
[ Info: Arnoldi iteration step 6: normres = 0.36355612628528766
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.15e-02, 1.41e-01, 3.13e-01, 9.34e-02, 4.37e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08537043732432295
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.09e-04, 1.75e-02, 1.95e-02, 3.93e-02, 6.34e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11799402273177732
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.62e-